In [ ]:
from pydub import AudioSegment
from pydub.silence import detect_nonsilent
#import moviepy.editor as mp
import noisereduce as nr
import numpy as np
import librosa
from moviepy import * as mp
# Step 1: Extract Audio from Video
def extract_audio_from_video(video_path, audio_path):
    video = mp.VideoFileClip(video_path)
    video.audio.write_audiofile(audio_path)

# Step 2: Remove Noise from Audio
def reduce_noise(audio_path, output_path):
    # Load audio file
    audio, sr = librosa.load(audio_path, sr=None)

    # Apply noise reduction
    reduced_audio = nr.reduce_noise(y=audio, sr=sr, y_noise=audio[:sr*1])  # Assuming first 1 second is noise

    # Save denoised audio
    librosa.output.write_wav(output_path, reduced_audio, sr)

# Step 3: Remove Silence from Audio
def remove_silence(audio_path, output_path, silence_thresh=-40, min_silence_len=1000):
    # Load audio file
    audio = AudioSegment.from_file(audio_path, format="wav")

    # Detect non-silent chunks
    nonsilent_chunks = detect_nonsilent(audio, min_silence_len=min_silence_len, silence_thresh=silence_thresh)

    # Combine non-silent chunks
    nonsilent_audio = AudioSegment.silent(duration=0)
    for start, end in nonsilent_chunks:
        nonsilent_audio += audio[start:end]

    # Export the result
    nonsilent_audio.export(output_path, format="wav")

# Step 4: Replace Original Audio in Video
def replace_audio_in_video(video_path, audio_path, output_video_path):
    video = mp.VideoFileClip(video_path)
    audio = mp.AudioFileClip(audio_path)
    video = video.set_audio(audio)
    video.write_videofile(output_video_path, codec="libx264", audio_codec="aac")

# Main Function
def process_video(video_path, output_video_path):
    temp_audio_path = "temp_audio.wav"
    denoised_audio_path = "denoised_audio.wav"
    nonsilent_audio_path = "nonsilent_audio.wav"

    # Extract audio from video
    extract_audio_from_video(video_path, temp_audio_path)

    # Reduce noise in audio
    reduce_noise(temp_audio_path, denoised_audio_path)

    # Remove silence from audio
    remove_silence(denoised_audio_path, nonsilent_audio_path)

    # Replace audio in video
    replace_audio_in_video(video_path, nonsilent_audio_path, output_video_path)

# Example Usage
video_input = "Unit-2 TA Session 1.mp4"
video_output = "output_video.mp4"
process_video(video_input, video_output)


NameError: name 'mp' is not defined